In [1]:
from cerebralcortex import Kernel
import json
# from IPython.display import display
# import cufflinks as cf
from plotly.offline import iplot, init_notebook_mode
# import plotly.graph_objs as go
# import plotly.figure_factory as ff
import numpy as np
import pandas as pd
from datetime import datetime, timedelta
import random
from cerebralcortex.core.datatypes import DataStream
from cerebralcortex.core.metadata_manager.stream.metadata import Metadata, DataDescriptor, ModuleMetadata
from cerebralcortex.core.util.spark_helper import get_or_create_sc
import ipywidgets as widgets


init_notebook_mode(connected=True)
pd.set_option('display.max_rows',5)

In [2]:
import pyspark.sql.functions as F
from pyspark.sql.types import *

In [3]:
CC = Kernel("/home/jupyter/cc3_conf/", study_name='mperf')

In [4]:
CC.search_stream('gyro')

['/cc3/study=mperf/gyroscope--org.md2k.motionsense--motion_sense_hrv--left_wrist',
 '/cc3/study=mperf/gyroscope--org.md2k.motionsense--motion_sense_hrv--right_wrist',
 '/cc3/study=mperf/gyroscope--org.md2k.phonesensor--phone']

In [5]:
accelerometer_right_wrist_stream_name = 'accelerometer--org.md2k.motionsense--motion_sense_hrv--right_wrist'
gyroscope_right_wrist_stream_name = 'gyroscope--org.md2k.motionsense--motion_sense_hrv--right_wrist'

In [6]:
user_id = CC.list_users()[0]['user_id']

In [7]:
accel_ds = CC.get_stream(accelerometer_right_wrist_stream_name).filter_user(user_id).filter("accelerometer_x >= -4")\
                    .filter("accelerometer_x <= 4").filter("accelerometer_y >= -4").filter("accelerometer_y <= 4")\
                    .filter("accelerometer_z >= -4").filter("accelerometer_z <= 4")

In [8]:
accel_ds.show(5)

Py4JJavaError: An error occurred while calling o132.showString.
: org.apache.spark.SparkException: Job aborted due to stage failure: Task 19 in stage 4.0 failed 4 times, most recent failure: Lost task 19.3 in stage 4.0 (TID 443, 10.40.40.73, executor 2): ExecutorLostFailure (executor 2 exited caused by one of the running tasks) Reason: Remote RPC client disassociated. Likely due to containers exceeding thresholds, or network issues. Check driver logs for WARN messages.
Driver stacktrace:
	at org.apache.spark.scheduler.DAGScheduler.failJobAndIndependentStages(DAGScheduler.scala:1979)
	at org.apache.spark.scheduler.DAGScheduler.$anonfun$abortStage$2(DAGScheduler.scala:1967)
	at org.apache.spark.scheduler.DAGScheduler.$anonfun$abortStage$2$adapted(DAGScheduler.scala:1966)
	at scala.collection.mutable.ResizableArray.foreach(ResizableArray.scala:62)
	at scala.collection.mutable.ResizableArray.foreach$(ResizableArray.scala:55)
	at scala.collection.mutable.ArrayBuffer.foreach(ArrayBuffer.scal

In [7]:
gyro_ds = CC.get_stream(gyroscope_right_wrist_stream_name).filter_user(user_id).filter("gyroscope_x >= -250")\
                    .filter("gyroscope_x <= 250").filter("gyroscope_y >= -250").filter("gyroscope_y <= 250")\
                    .filter("gyroscope_z >= -250").filter("gyroscope_z <= 250")

In [8]:
accel_w = accel_ds.window(windowDuration=60)
gyro_w = gyro_ds.window(windowDuration=60)

In [10]:
gyro_w = gyro_w.drop("user", "version")

In [11]:
common_ds = accel_w.join(gyro_w)

In [14]:
common_ds.show(5)

+--------------------+-------+--------------------+--------------------+--------------------+--------------------+--------------------+--------------------+--------------------+--------------------+
|                user|version|              window|     accelerometer_y|     accelerometer_x|     accelerometer_z|              window|         gyroscope_z|         gyroscope_y|         gyroscope_x|
+--------------------+-------+--------------------+--------------------+--------------------+--------------------+--------------------+--------------------+--------------------+--------------------+
|f29a8628-ed1d-445...|      1|[2017-11-11 20:25...|[0.0665283203125,...|[-0.070556640625,...|[1.013671875, 1.0...|[2017-11-11 20:25...|[1.953125, 1.9683...|[0.4119873046875,...|[-1.5411376953125...|
|f29a8628-ed1d-445...|      1|[2017-11-11 20:25...|[0.0665283203125,...|[-0.070556640625,...|[1.013671875, 1.0...|[2017-11-12 08:55...|[2.532958984375, ...|[0.3814697265625,...|[-1.46484375, -1....|
|f29a

In [15]:
print("finished")

finished


In [19]:
def is_correct_accel_data(accel_x, accel_y, accel_z):
    data = np.array([accel_x, accel_y, accel_z]).T
    mg = np.mean([np.linalg.norm(x) for x in data])
    if mg < 0.8:
        return 0
    return 1
udf_is_correct_accel_data = F.udf(is_correct_accel_data, IntegerType())

def correct_accel(data, is_correct):
    if is_correct == 1:
        return data
    data = np.array(data)*2
    return data.tolist()
udf_correct_accel = F.udf(correct_accel, ArrayType(DoubleType()))
